In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import re
import logging
import seaborn as sns
import pandas as pd
import awswrangler as wr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from general_functions.conncet_s3 import S3Connection
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url, return_service_token
from general_functions.call_api_with_account_id import send_to_innkeepr_api_paginated, call_api_with_accountId

In [3]:
tenant = "Junglueck"
audience ="66d17985c98dbc05719f6ebe"
tenant = "Rosental"
audience = "678a5ca0f3abb5123264a902"
url = return_api_url()
service_token = return_service_token()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == tenant]
account_id = account_id[0]
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/SprintStories/EN-2669-incident-action-probs/"

In [4]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=45)).strftime('%Y-%m-%d')
models = call_api_with_accountId(
    f"{url}/models/query",
    account_id, {
    "created": {"$lt": today, "$gte": start_date},
    "audienceId": audience
}, logging)
models=pd.json_normalize(models)
models.sort_values(by="created")

In [5]:
import re
def return_dates(text):
    print(text)
    match = re.findall(r'\d{4}-\d{2}-\d{2}', text)
    return match[-1]

In [6]:
#TODO: add properties in the future
if len(models) < 1:
    raise ValueError(f"Time window is not long enough for incident handling")
models = models.sort_values(by="created").reset_index(drop=True)
max_model_created = models["created"].max()
print(f"max_model_created: {max_model_created}")
model_current = models[models["created"] == max_model_created]
s3 = S3Connection()
tenant_sanitized = tenant.replace(" ", "").lower()
bucket = f"test-innkeepr-targeting-{tenant_sanitized}"
print(f"bucket = {bucket}")
for imodel, model in model_current.iterrows():
    print(f"{imodel}: {model.created} - {model.id}")
    path = model.path
    current_path_date = return_dates(path)
    files = s3.list_files_with_pagination(bucket, path)
    file_date = return_dates(files[0])
    if file_date < current_path_date:
        print(f"Switched to previous model: {file_date} < {current_path_date}")
        switch_to_previous_model = True
    else:
        print(f"Other switch: {file_date} >= {current_path_date}")
        switch_to_previous_model = False
print(f"switch_to_previous_model = {switch_to_previous_model}")
if switch_to_previous_model == False:
    #if switch is true
    # label accordingly
    # if switch is false
    # query old model
    today = datetime.today().strftime('%Y-%m-%d')
    models_prev = models.iloc[models.index[-2]]
    model_id = models_prev.id
    model_properties = call_api_with_accountId(
        f"{url}/models/query",
        account_id, {
        "id": model_id
    }, logging)
    if len(model_properties) != 1:
        raise ValueError(f"More than one model found: {model_properties}")
    model_properties = model_properties[0]
    print(f"model_properties = {model_properties}")
    # copy files to new model
    models_prev_path = models_prev.path
    date_prev_path = return_dates(models_prev_path)
    print(f"date_prev_path = {date_prev_path}, models_prev_path = {models_prev_path}")
    new_path = models_prev_path.replace(date_prev_path, today)
    print(f"new_path = {new_path}")
    s3.copy_all_files_recursively(bucket, models_prev_path, new_path)
    coding_prev_path = f"{models_prev_path.split('_best_models')[0]}-coding"
    coding_path_new = coding_prev_path.replace(date_prev_path, today)
    s3.copy_all_files_recursively(bucket, coding_prev_path, coding_path_new)
    # update model
    new_model_properties = model_properties
    new_model_properties["path"] = new_path
    new_model_properties["created"] = datetime.now().isoformat() + "Z"
    new_model_properties["manualRetrainReason"] = "error_type"#change production 
    new_model_properties.pop("id")
    # Endpoint to update model
    print(f"new_model_properties = {new_model_properties}")
    model_upate = {"id":"testid"} #TODO: change
    new_model_id = model_upate["id"] #change production 
    # update audience
    payload = {
        "id":audience,
        "config":{"model": new_model_id}
    }
    print(f"Update Audience with model_upate = {payload}")

In [7]:
models

In [13]:
#if switch is true
# label accordingly
# if switch is false
# query old model
today = datetime.today().strftime('%Y-%m-%d')
models_prev = models.iloc[models.index[-2]]
model_id = models_prev.id
model_properties = call_api_with_accountId(
    f"{url}/models/query",
    account_id, {
    "id": model_id
}, logging)
if len(model_properties) != 1:
    raise ValueError(f"More than one model found: {model_properties}")
model_properties = model_properties[0]
print(f"model_properties = {model_properties}")
# copy files to new model
models_prev_path = models_prev.path
date_prev_path = return_dates(models_prev_path)
print(f"date_prev_path = {date_prev_path}, models_prev_path = {models_prev_path}")
new_path = models_prev_path.replace(date_prev_path, today)
print(f"new_path = {new_path}")
s3.copy_all_files_recursively(bucket, models_prev_path, new_path)
coding_prev_path = f"{models_prev_path.split('_best_models')[0]}-coding"
coding_path_new = coding_prev_path.replace(date_prev_path, today)
s3.copy_all_files_recursively(bucket, coding_prev_path, coding_path_new)
# update model
new_model_properties = model_properties
new_model_properties["path"] = new_path
new_model_properties["created"] = datetime.now().isoformat() + "Z"
new_model_properties["manualRetrainReason"] = "error_type"#change production 
new_model_properties.pop("id")
# Endpoint to update model
print(f"new_model_properties = {new_model_properties}")
model_upate = {"id":"testid"} #TODO: change
new_model_id = model_upate["id"] #change production 
# update audience
payload = {
    "id":audience,
    "config":{"model": new_model_id}
}
print(f"Update Audience with model_upate = {payload}")

In [ ]:
# Create Test Audience
